# TMC429 and TMC2130 Testing

## Setup output pins and f_clk for TMC429

In [1]:
import spidev
import pigpio
import os
from RpiMotorLib import RpiMotorLib
import json



TMC429_CS_PIN = 5   # GPIO 5
TMC429_CLK = 4      # GPIO 4
TMC2130_CS_PIN = 8  # GPIO 8
f_clk = 16000000


gpiod = pigpio.pi()


gpiod.set_mode(TMC429_CS_PIN, pigpio.OUTPUT)
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)

gpiod.set_mode(TMC429_CLK, pigpio.OUTPUT)
gpiod.hardware_clock(TMC429_CLK, f_clk)


gpiod.set_mode(TMC2130_CS_PIN, pigpio.OUTPUT)
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)

def parse_tmc429_status_byte(byte):
    status = {
        "INT": (byte & 0x80) != 0,
        "CDGW": (byte & 0x40) != 0,
        "RS3": (byte & 0x20) != 0,
        "RS2": (byte & 0x08) != 0,
        "RS1": (byte & 0x02) != 0,
        "xEQt3": (byte & 0x10) != 0,
        "xEQt2": (byte & 0x04) != 0,
        "xEQt1": (byte & 0x01) != 0
    }
    status = json.dumps(status, indent=2)
    print(status)

def parse_tmc2130_status(byte):
    print(f"'reset_flag':   {(byte & 0x01) != 0}")
    print(f"'driver_error': {(byte & 0x02) != 0}")
    print(f"'sg2':          {(byte & 0x04) != 0}")
    print(f"'standstill':   {(byte & 0x08) != 0}")

## Setup SPI bus

In [2]:
import spidev
import time

SPI_BUS = 0
SPI_DEVICE = 0
spi = spidev.SpiDev()
spi.open(SPI_BUS, SPI_DEVICE)
spi.max_speed_hz = 100000
spi.no_cs = True
spi.mode = 3





# TMC429 Commands

## Read version and get status

In [4]:
# TMC 429
# Read the TMC 429 version
register = 0b01110011

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")
print(parse_tmc429_status_byte(response[0]))

0x14 0x42 0x91 0x1
{
  "INT": false,
  "CDGW": false,
  "RS3": false,
  "RS2": false,
  "RS1": false,
  "xEQt3": true,
  "xEQt2": true,
  "xEQt1": false
}
None


## Read pulse_div and ramp_div

In [31]:
# TMC 429
# Read the TMC 429 Pulse_div and ramp_div
register = 0b00011001

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")

pulse_div = (response[2] & 0xF0) >> 4
ramp_div = (response[2] & 0xF)
print(f"pulse_div: {pulse_div}")
print(f"ramp_div:  {ramp_div}")

# Read the TMC 429 PMUL and PDIV
register = 0b000010011

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")

pmul = (response[2])
pdiv = (response[3])
print(f"pmul: {pmul}")
print(f"pdiv: {pdiv}")

0x14 0x0 0x75 0x0
pulse_div: 7
ramp_div:  5
0x14 0x0 0xfa 0x0
pmul: 250
pdiv: 0


## Set pulse_div, ramp_div, pmul and pdiv

In [30]:
# TMC 429
# Set the TMC 429 Pulse_div and ramp_div
register = 0b00011000
byte3 = 0b00000000
byte2 = 0b01110101
byte1 = 0b00000000

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")

# Set the TMC 429 PMUL and PDIV
register = 0b000010010
byte3 = 0b00000000
byte2 = 0b11111010
byte1 = 0b00000000

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")


0x14 0x0 0x0 0x0
0x14 0x0 0x0 0x0


## Read Velocities

In [29]:
# TMC 429
# Read the TMC 429 Target Speed
register = 0b00001001

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"Tgt Speed: {hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")


# Read the TMC 429 Min Speed
register = 0b00000101

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"Min Speed: {hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")


# Read the TMC 429 Max Speed
register = 0b00000111

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"Max Speed: {hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")


# Read the TMC 429 Max Accel
register = 0b00001101

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"Max Accel: {hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")

Tgt Speed: 0x14 0x0 0x3 0x46
Min Speed: 0x14 0x0 0x0 0x0
Max Speed: 0x14 0x0 0x7 0xff
Max Accel: 0x14 0x0 0x3 0xe8


## Set Velocities

In [76]:
# TMC 429
# Set the TMC 429 Target Speed

trgt_speed = 0
# trgt_speed = 838 # 1 RPS with 5 pulse_div
# trgt_speed = 45  # Siderial rate with 7 pulse_div

if trgt_speed > 0x7ff:
    trgt_speed = 0x7ff

register = 0b00001000
byte3 = 0b00000000
byte2 = trgt_speed >> 8
byte1 = trgt_speed & 0xFF

print(bin(byte2))
print(bin(byte1))


print(f"Max speed: {f_clk / (2**pulse_div * 32)}")

gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)

print(f"Tgt Speed: {float((f_clk*  trgt_speed) / ((2**pulse_div) * 2048 * 32)) }")
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")


# Set the TMC 429 Max Speed
register = 0b00000110
byte3 = 0b00000000
byte2 = 0x07
byte1 = 0xff

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")

# Set the TMC 429 Min Speed
register = 0b00000100

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")

# Set the TMC 429 Max Accel
register = 0b00001100
byte3 = 0b00000000
byte2 = 0x00
byte1 = 0x10

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")

0b0
0b0
Max speed: 3906.25
Tgt Speed: 0.0
0x14 0x0 0x0 0x0
0x14 0x0 0x0 0x0
0x14 0x0 0x0 0x0
0x14 0x0 0x0 0x0


## Set Velocity Mode

In [74]:
# TMC 429
# Set the TMC 429 to velocity mode
register = 0b00010100
byte3 = 0b00000000
byte2 = 0b00000000      # Disable endstop L and R
byte1 = 0b00000010      # Velocity mode

gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)

register = 0b00010101
# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{bin(response[0])} {bin(response[1])} {bin(response[2])} {bin(response[3])}")

0b10100 0b0 0b0 0b10


## Set Ramp Mode

In [44]:
# TMC 429
# Set the TMC 429 to velocity mode
register = 0b00010100
byte3 = 0b00000000
byte2 = 0b00000000      # Disable endstop L and R
byte1 = 0b00000000      # Velocity mode

gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)

register = 0b00010101
# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{bin(response[0])} {bin(response[1])} {bin(response[2])} {bin(response[3])}")

0b10100 0b0 0b0 0b0


## Set Hold Mode

In [43]:
# TMC 429
# Set the TMC 429 to velocity mode
register = 0b00010100
byte3 = 0b00000000
byte2 = 0b00000000      # Disable endstop L and R
byte1 = 0b00000011      # Velocity mode

gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)

register = 0b00010101
# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{bin(response[0])} {bin(response[1])} {bin(response[2])} {bin(response[3])}")

0b10100 0b0 0b0 0b11


## Read IF Config

In [12]:
# TMC 429
# Read the TMC 429 IF Config
register = 0b01101001

gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{bin(response[0])} {bin(response[1])} {bin(response[2])} {bin(response[3])}")

0b10100 0b0 0b0 0b100000


## Set Step/Dir mode

In [13]:
# TMC 429
# Set the TMC 429 in Step and Dir mode
register = 0b01101000
byte3 = 0b00000000
byte2 = 0b00000000
byte1 = 0b00100000

gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{bin(response[0])} {bin(response[1])} {bin(response[2])} {bin(response[3])}")

0b10100 0b0 0b0 0b0


# Calculate pules_dive and velocity for given PPS

In [234]:
import sys
# Calculate velocity and pulse_dev for given step_pulse_rate

def distance_to_nearest_round_number(x):
    # Find the nearest integer by rounding
    nearest_round_number = round(x)
    # Calculate the absolute difference between the float and the nearest round number
    distance = abs(x - nearest_round_number)
    return distance

f_clk = 16000000 # Hz
# step_pulse_rate = 40960 # Hz GoTo
step_pulse_rate = 85.96 # Hz Siderial
# step_pulse_rate = 6400 # Hz 
pulse_dev = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
prev_error = 1.0
smallest_error = {
    "pulse_dev": None,
    "velocity": None,
    "error": None
}
for number in pulse_dev:
    velocity = (step_pulse_rate * 2048 * 32 * 2**number) / (f_clk)
    error = abs(velocity - round(velocity))
    if error < prev_error:
        prev_error = error
        smallest_error["pulse_dev"] = number
        smallest_error["velocity"] = int(velocity)
        smallest_error["error"] = error
    print(f"For pulse_div: {number} velocity is {round(velocity, 2)}  error: {error}")
    if velocity > 2048:
        print("Velocity is over 2048. Stopping")
        break

new_step_pulse_rate = (f_clk * smallest_error["velocity"] ) / (2048 * 32 * 2**smallest_error['pulse_dev'])
print(f"\nSmallest error is for pulse_div: {smallest_error['pulse_dev']} with velocity {smallest_error['velocity']} and error {smallest_error['error']}")
print(f"New step_pulse_rate = {new_step_pulse_rate} Hz with error {abs(new_step_pulse_rate - step_pulse_rate)}")

For pulse_div: 0 velocity is 0.35  error: 0.35209216
For pulse_div: 1 velocity is 0.7  error: 0.29581568
For pulse_div: 2 velocity is 1.41  error: 0.40836863999999995
For pulse_div: 3 velocity is 2.82  error: 0.1832627200000001
For pulse_div: 4 velocity is 5.63  error: 0.3665254400000002
For pulse_div: 5 velocity is 11.27  error: 0.2669491199999996
For pulse_div: 6 velocity is 22.53  error: 0.4661017600000008
For pulse_div: 7 velocity is 45.07  error: 0.06779647999999838
For pulse_div: 8 velocity is 90.14  error: 0.13559295999999676
For pulse_div: 9 velocity is 180.27  error: 0.27118591999999353
For pulse_div: 10 velocity is 360.54  error: 0.45762816000001294
For pulse_div: 11 velocity is 721.08  error: 0.08474367999997412
For pulse_div: 12 velocity is 1442.17  error: 0.16948735999994824
For pulse_div: 13 velocity is 2884.34  error: 0.3389747199998965
Velocity is over 2048. Stopping

Smallest error is for pulse_div: 7 with velocity 45 and error 0.06779647999999838
New step_pulse_rate =

# TMC 2130 Commands

## Read veresion and get status

In [5]:
ADDR_GCONF = 0x00
ADDR_IHOLD_IRUN = 0x10
ADDR_TPOWERDOWN = 0x11
ADDR_TPWMTHRS = 0x13
ADDR_TCOOLTHRS = 0x14
ADDR_THIGH = 0x15
ADDR_XDIRECT = 0x2D
ADDR_VDCMIN = 0x33
ADDR_CHOPCONF = 0x6C
ADDR_COOLCONF = 0x6D
ADDR_PWMCONF = 0x70
ADDR_ENCM_CTRL = 0x72


register = 0x04

# Sending the read command
# Send the read command two times on the TMC2130
gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0, 0])
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)
gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0, 0])
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)
# print(response)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])} {hex(response[4])}")
parse_tmc2130_status(response[0])
print(f"'STEP':         {(response[4] & 0x01) != 0}")
print(f"'DIR':          {(response[4] & 0x02) != 0}")
print(f"'DCEN_CFG4':    {(response[4] & 0x04) != 0}")
print(f"'DCEN_CFG6':    {(response[4] & 0x08) != 0}")
print(f"'DRV_ENN_CFG6': {(response[4] & 0x10) != 0}")
print(f"'DCO':          {(response[4] & 0x20) != 0}")
print(f"'VERSION':      {hex(response[1])}")

0x30 0x11 0x0 0x0 0x68
'reset_flag':   False
'driver_error': False
'sg2':          False
'standstill':   False
'STEP':         False
'DIR':          False
'DCEN_CFG4':    False
'DCEN_CFG6':    True
'DRV_ENN_CFG6': False
'DCO':          True
'VERSION':      0x11


## Read GSTAT register

- resets `reset_flag`

In [63]:
# TMC 2130
register = 0x01

# Sending the read command
# Send the read command two times on the TMC2130
gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0, 0])
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)
gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0, 0])
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)
# print(response)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])} {hex(response[4])}")
parse_tmc2130_status(response[0])
print(f"'reset':        {(response[4] & 0x01) != 0}")
print(f"'drv_err':      {(response[4] & 0x02) != 0}")
print(f"'uv_cp':        {(response[4] & 0x04) != 0}")

0x39 0x0 0x0 0x0 0x1
'reset_flag':   True
'driver_error': False
'sg2':          False
'standstill':   True
'reset':        True
'drv_err':      False
'uv_cp':        False


## Reset Driver

In [106]:


config_addresses = [
    ADDR_GCONF,
    ADDR_IHOLD_IRUN,
    ADDR_TPOWERDOWN,
    ADDR_TPWMTHRS,
    ADDR_TCOOLTHRS,
    ADDR_THIGH,
    ADDR_XDIRECT,
    ADDR_VDCMIN,
    ADDR_CHOPCONF,
    ADDR_COOLCONF,
    ADDR_PWMCONF,
    ADDR_ENCM_CTRL
]

for address in config_addresses:
    write_address = address | (1 << 8)
    gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
    response = spi.xfer2([write_address, 0, 0, 0, 0])
    gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)
    

## Setup Driver

In [68]:
# Chopper config
# TOFF = 4
# HSTRT = 6
# HEND = 0
# CHM = 0
# TBL = 2
# Microsteps = 3 / 32x

register = ADDR_CHOPCONF
byte1 = 0b00000011
byte2 = 0b00000001
byte3 = 0b10000000
byte4 = 0b01100100

write_address = register + 0x80
print(bin(write_address))
print(bin(register))
gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
response = spi.xfer2([write_address, byte1, byte2, byte3, byte4])
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)

parse_tmc2130_status(response[0])

# PWM Config
# ampl = 200
# pwm_grad = 4
# pwm_freq = 0
# pwm_autoscale = 1

register = ADDR_PWMCONF
byte1 = 0b00000000
byte2 = 0b00000100
byte3 = 0b00000100
byte4 = 0b11001000

write_address = register + 0x80
print(bin(write_address))
print(bin(register))
gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
response = spi.xfer2([write_address, byte1, byte2, byte3, byte4])
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)

parse_tmc2130_status(response[0])

0b11101100
0b1101100
'reset_flag':   False
'driver_error': False
'sg2':          False
'standstill':   True
0b11110000
0b1110000
'reset_flag':   False
'driver_error': False
'sg2':          False
'standstill':   True


## Enable Stealth Chop

In [66]:
register = ADDR_GCONF
byte1 = 0b00000000
byte2 = 0b00000000
byte3 = 0b00000000
byte4 = 0b00000100

write_address = register + 0x80
print(bin(write_address))
print(bin(register))
gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
response = spi.xfer2([write_address, byte1, byte2, byte3, byte4])
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)

parse_tmc2130_status(response[0])



0b10000000
0b0
'reset_flag':   False
'driver_error': False
'sg2':          False
'standstill':   True


## Set Hold and Run Current

In [73]:
hold_current = 5 # %
run_current = 0 # %

register = 0x10
byte1 = 0b00000000
byte2 = 0b00000000
byte3 = 0b00000000
byte4 = 0b00000000 + hold_current + (run_current << 4)

write_address = register + 0x80
print(bin(byte4))
print(bin(write_address))
print(bin(register))
gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
response = spi.xfer2([write_address, byte1, byte2, byte3, byte4])
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)

print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])} {hex(response[4])}")
parse_tmc2130_status(response[0])

0b101
0b10010000
0b10000
0x38 0x0 0x0 0x0 0xa
'reset_flag':   False
'driver_error': False
'sg2':          False
'standstill':   True


## Get CHOPCONF

In [227]:
register = ADDR_CHOPCONF

gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0, 0])
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)

gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0, 0])
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)

print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])} {hex(response[4])}")
parse_tmc2130_status(response[0])

0x38 0x0 0x0 0x0 0x0
'reset_flag':   False
'driver_error': False
'sg2':          False
'standstill':   True


## Get PWMCONF

In [226]:
register = ADDR_PWMCONF

gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0, 0])
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)

gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0, 0])
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)

print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])} {hex(response[4])}")
parse_tmc2130_status(response[0])

0x38 0x0 0x0 0x0 0x0
'reset_flag':   False
'driver_error': False
'sg2':          False
'standstill':   True


Set CHOPCONF

In [221]:
register = ADDR_CHOPCONF
byte1 = 0b00000000  # 32x microstep
byte2 = 0b00000000
byte3 = 0b00000000
byte4 = 0b00000000

write_address = register + 0x80
print(bin(write_address))
print(bin(register))

gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
response = spi.xfer2([write_address, byte1, byte2, byte3, byte4])
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)

print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])} {hex(response[4])}")
parse_tmc2130_status(response[0])

0b11101100
0b1101100
0x70 0x0 0x0 0x0 0x0
'reset_flag':   False
'driver_error': False
'sg2':          False
'standstill':   False
